# Solving Equations with Julia-Defined Types

One of the nice things about DifferentialEquations.jl is that it is designed with Julia's type system in mind. What this means is, if you have properly defined a Number type, you can use this number type in DifferentialEquations.jl's algorithms! [Note that this is restricted to the native algorithms of DifferentialEquations.jl. The external solvers are not compatible with some number systems. For example, ODE.jl will throw errors unless certain options are set, and ODEInterface will convert the numbers to floats].

DifferentialEquations.jl determines the numbers to use in its solvers via the types that are designated by Δt and the initial condition of the problem. It will keep the time values in the same type as Δt, and the solution values in the same type as the initial condition. [Note that adaptive timestepping requires that Δt be compaible with `sqrt` and `^` functions. Thus Δt cannot be Integer or numbers like that if adaptive timestepping is chosen].

Let's solve the linear ODE first define an easy way to get ODEProblems for the linear ODE:

In [2]:
using DifferentialEquations
f = (t,u) -> (1.01*u)
analytic = (t,u₀) -> u₀*exp(1.01*t)
"""Linear ODE on Float64"""
prob_ode_linear = ODEProblem(f,1/2,analytic=analytic);

INFO: Recompiling stale cache file /home/crackauc/.julia/lib/v0.5/DifferentialEquations.ji for module DifferentialEquations.
 in depwarn(::String, ::Symbol) at ./deprecated.jl:64
 in symbol(::String, ::Vararg{String,N}) at ./deprecated.jl:30
 in replace_t(::Expr) at /home/crackauc/.julia/v0.5/InplaceOps/src/InplaceOps.jl:97
 in collect_to!(::Array{Expr,1}, ::Base.Generator{Array{Any,1},InplaceOps.#replace_t}, ::Int64, ::Int64) at ./array.jl:337
 in collect(::Base.Generator{Array{Any,1},InplaceOps.#replace_t}) at ./array.jl:305
 in @into!(::Any) at /home/crackauc/.julia/v0.5/InplaceOps/src/InplaceOps.jl:120
 in macro expansion; at ./none:2 [inlined]
 in anonymous at ./<missing>:?
while loading /home/crackauc/.julia/v0.5/DifferentialEquations/src/ode/ode_integrators.jl, in expression starting on line 4039


First let's solve it using Float64s. To do so, we just need to set u₀ to a Float64 (which is done by the default) and Δt should be a float as well.

In [2]:
prob = prob_ode_linear 
sol =solve(prob::ODEProblem,[0,1],Δt=1/2^(6),save_timeseries=true,alg=:RK4)
println(sol)

DifferentialEquations.ODESolution with 65 timesteps. Analytical solution is known.
u: 1.3728005068011597
errors: Dict(:l∞=>7.07298e-10,:final=>7.07298e-10,:l2=>3.31617e-10)
t: [0.0,0.015625,0.03125,0.046875,0.0625,0.078125,0.09375,0.109375,0.125,0.140625,0.15625,0.171875,0.1875,0.203125,0.21875,0.234375,0.25,0.265625,0.28125,0.296875,0.3125,0.328125,0.34375,0.359375,0.375,0.390625,0.40625,0.421875,0.4375,0.453125,0.46875,0.484375,0.5,0.515625,0.53125,0.546875,0.5625,0.578125,0.59375,0.609375,0.625,0.640625,0.65625,0.671875,0.6875,0.703125,0.71875,0.734375,0.75,0.765625,0.78125,0.796875,0.8125,0.828125,0.84375,0.859375,0.875,0.890625,0.90625,0.921875,0.9375,0.953125,0.96875,0.984375,1.0]
timeseries: [0.5,0.507953,0.516033,0.524241,0.53258,0.541051,0.549658,0.558401,0.567283,0.576306,0.585473,0.594786,0.604247,0.613858,0.623623,0.633542,0.64362,0.653857,0.664258,0.674824,0.685558,0.696463,0.707541,0.718795,0.730229,0.741844,0.753644,0.765632,0.777811,0.790183,0.802752,0.815521,0.828493,0

Notice that both the times and the solutions were saved as Float64. Let's change the time to use rational values:

In [3]:
sol =solve(prob::ODEProblem,[0,1],Δt=1//2^(6),save_timeseries=true,alg=:RK4)
println(sol)

DifferentialEquations.ODESolution with 65 timesteps. Analytical solution is known.
u: 1.3728005068011597
errors: Dict(:l∞=>7.07298e-10,:final=>7.07298e-10,:l2=>3.31617e-10)
t: Rational{Int64}[0//1,1//64,1//32,3//64,1//16,5//64,3//32,7//64,1//8,9//64,5//32,11//64,3//16,13//64,7//32,15//64,1//4,17//64,9//32,19//64,5//16,21//64,11//32,23//64,3//8,25//64,13//32,27//64,7//16,29//64,15//32,31//64,1//2,33//64,17//32,35//64,9//16,37//64,19//32,39//64,5//8,41//64,21//32,43//64,11//16,45//64,23//32,47//64,3//4,49//64,25//32,51//64,13//16,53//64,27//32,55//64,7//8,57//64,29//32,59//64,15//16,61//64,31//32,63//64,1//1]
timeseries: [0.5,0.507953,0.516033,0.524241,0.53258,0.541051,0.549658,0.558401,0.567283,0.576306,0.585473,0.594786,0.604247,0.613858,0.623623,0.633542,0.64362,0.653857,0.664258,0.674824,0.685558,0.696463,0.707541,0.718795,0.730229,0.741844,0.753644,0.765632,0.777811,0.790183,0.802752,0.815521,0.828493,0.841671,0.855059,0.86866,0.882477,0.896514,0.910775,0.925262,0.93998,0.954931,0.9

Now let's do something fun. Let's change the solution to use `Rational{BigInt}` and print out the value at the end of the simulation. To do so, simply change the definition of the initial condition. 

In [3]:
const linear_bigα = parse(BigFloat,"1.01")
f = (t,u) -> (linear_bigα*u)
analytic = (t,u₀) -> u₀*exp(linear_bigα*t)
"""Linear ODE on Float64"""
prob_ode_bigfloatlinear = ODEProblem(f,parse(BigFloat,"0.5"),analytic=analytic)
prob = prob_ode_bigfloatlinear
sol =solve(prob::ODEProblem,[0,1],Δt=1//2^(6),save_timeseries=true,alg=:RK4)
println(sol[end])

1.372800506801159633066911929316347253809948567877000660264442201258269308188093


That's one huge fraction! What about the analytical solution?

In [5]:
sol.timeseries_analytic[end]

1.372800507508458246994888158330193812036875409797981145833699043993648584121963

This is to be expected. Notice that when we defined `analytic`, we used the `exp` function. In Julia, this is defined on `Rational{BigInt}` to spit out a `BigFloat`, and so all of the analytical solution's values change to `BigFloat` to compensate. This shows that DifferentialEquations.jl is using the correct numbers. So can we do more?

## Other Compatible Number Types

#### ArbFloats

Let's test a bunch of other number types. First I'm going to test Jeffrey Sarnoff's ArbFloats. These high precision numbers which are much faster than Bigs for less than 500-800 bits of accuracy. Having already installed Nemo and ArbFloats, I can use them in DifferentialEquations.jl via:

NOTE: 9/12/2016 for the newest version, this needs a small PR to go through on ArbFloats. Pending.

In [4]:
using ArbFloats
const arbalpha = ArbFloat(1.01)
f = (t,u) -> (arbalpha*u)
analytic = (t,u₀) -> u₀*exp(arbalpha*t)
prob_ode_arbfloatlinear = ODEProblem(f,ArbFloat(1)/ArbFloat(2),analytic=analytic)
sol =solve(prob_ode_arbfloatlinear::ODEProblem,[0,1],Δt=1//2^(6),save_timeseries=true,alg=:RK4)
println(sol[end])

1.37280050680115964525982974524059


Let's double-check that value:

In [5]:
typeof(sol[end])

ArbFloats.ArbFloat{116}

Bingo! ArbFloats work with DifferentialEquations.jl.

#### DecFP.jl

Next let's try DecFP. DecFP is a fixed-precision decimals library which is made to give both performance but known decimals of accuracy. Having already installed DecFP with `Pkg.add("DecFP")`, I can run the following:

In [11]:
using DecFP
const decalpha = Dec128(1.01) # Set the constant to a Dec128 as well
f = (t,u) -> (decalpha*u)
analytic = (t,u₀) -> u₀*exp(decalpha*t)
prob_ode_decfplinear = ODEProblem(f,Dec128(1)/Dec128(2),analytic=analytic)
sol =solve(prob_ode_decfplinear::ODEProblem,[0,1],Δt=1//2^(6),save_timeseries=true,alg=:RK4)
println(sol[end]); println(typeof(sol[end]))

+1372800506801159645259829745240592E-33
DecFP.Dec128


Bingo! DecFP works with DifferentialEquations.jl

## Incompatible Number Systems

#### Decimals.jl

Install with `Pkg.add("Decimals")`.

In [ ]:
using Decimals
prob_ode_decimallinear = ODEProblem(f,[decimal("1.0")]./[decimal("2.0")],analytic=analytic)
sol =solve(prob_ode_decimallinear::ODEProblem,[0,1],Δt=1/2^(6),save_timeseries=true,alg=:RK4) #Fails
println(sol[end]); println(typeof(sol[end]))

ERROR (unhandled task failure): MethodError: no method matching show_backtrace(::Base.AbstractIOBuffer{Array{UInt8,1}}, ::Symbol, ::Array{Ptr{Void},1}, ::UnitRange{Int64})
Closest candidates are:
  show_backtrace(::IO, !Matched::Array{Any,1}) at replutil.jl:577
  show_backtrace(::IO, !Matched::Array{T,1}) at replutil.jl:571


At the time of writing this, Decimals are not compatible. This is not on DifferentialEquations.jl's end, it's on Decimal's end since it stems from not having an `abs` function (along with many other functions being missing!). Thus it's not recommended you use Decimals with DifferentialEquations.jl

#### ArbReals.jl

A fast number system using intervals which is still in development. These are interval versions of ArbReals designed to give checked-arithmetic. See the repo for the current install instructions.

In [ ]:
using ArbReals
prob_ode_arbrealslinear = ODEProblem(f,ArbReals(1)/ArbReals(2),analytic=analytic)
sol =solve(prob_ode_arbrealslinear::ODEProblem,[0,1],Δt=1//2^(6),save_timeseries=true,alg=:RK4)
println(sol[end]); println(typeof(sol[end]))

At the time of writing this I couldn't get ArbReals to precompile.

#### DoubleDouble.jl

Install via `Pkg.add("DoubleDouble")`

In [ ]:
using DoubleDouble
prob_ode_doublelinear = ODEProblem(f,Double(1)/Double(2),analytic=analytic)
sol =solve(prob_ode_doublelinear::ODEProblem,[0,1],Δt=1/2^(6),save_timeseries=true,alg=:RK4)
println(sol[end]); println(typeof(sol[end]))

ERROR (unhandled task failure): MethodError: no method matching show_backtrace(::Base.AbstractIOBuffer{Array{UInt8,1}}, ::Symbol, ::Array{Ptr{Void},1}, ::UnitRange{Int64})
Closest candidates are:
  show_backtrace(::IO, !Matched::Array{Any,1}) at replutil.jl:577
  show_backtrace(::IO, !Matched::Array{T,1}) at replutil.jl:571


DoubleDouble is erroring because DoubleDoubles cannot multiply Ints! [An issue has been filed to the DoubleDouble.jl repo for this case](https://github.com/simonbyrne/DoubleDouble.jl/issues/16). If you checkout the branch from the Issue, you will see that it will still error because DoubleDouble isn't compatible with `exp`. [There's another issue for that.](https://github.com/simonbyrne/DoubleDouble.jl/issues/8). 

## Conclusion

As you can see, DifferentialEquations.jl can use arbitrary Julia-defined number systems in its arithmetic. The only limitations are the limitations inherent in the number systems themselves. ArbFloats and ArbReals are the most feature-complete and give great performance compared to BigFloats, and thus I recommend their use when high-precision (less than ~512-800 bits) is required. DecFP is a great library for high-performance decimal numbers and works well as well. Other number systems could use some modernization.